In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#!pip install matplotlib

     ---------------------------------------- 7.2/7.2 MB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 55.3/55.3 kB 2.8 MB/s eta 0:00:00
     ------------------------------------- 163.6/163.6 kB 10.2 MB/s eta 0:00:00
     ------------------------------------- 965.4/965.4 kB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 12.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%matplotlib inline

In [8]:
import numpy as np
import pandas as pd

In [9]:
df_raw = pd.read_csv("network_df.csv").assign(
    owned_asset_count=lambda x: x.owned_asset_count.astype(float),
)
df_raw.info()
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188512 entries, 0 to 188511
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   dao_id             188512 non-null  object 
 1   member_address     188512 non-null  object 
 2   slug               188512 non-null  object 
 3   owned_asset_count  188512 non-null  float64
dtypes: float64(1), object(3)
memory usage: 5.8+ MB


,dao_id,member_address,slug,owned_asset_count
0,ec242c8a-4667-46c6-8c51-0df6f0970970,0x9533158bb9b0abb178a9dc7f6f0168e84126688f,movement-daos-ascended-ape-collection,1.0
1,f12e8672-59e5-425e-bfd9-42f4078a3ff1,0x097c39e5e576a8706404cd0d81e05b522f5bcaff,eth-merge-v3,1.0
2,f12e8672-59e5-425e-bfd9-42f4078a3ff1,0x097c39e5e576a8706404cd0d81e05b522f5bcaff,grand-leisure,1.0
3,f12e8672-59e5-425e-bfd9-42f4078a3ff1,0x097c39e5e576a8706404cd0d81e05b522f5bcaff,re-based-ghouls-by-xwave,1.0
4,f12e8672-59e5-425e-bfd9-42f4078a3ff1,0x097c39e5e576a8706404cd0d81e05b522f5bcaff,based-ghouls,6.0


In [11]:
# add DAO names
 import io, json

 with io.open("../../3_api/deepdao/data/deepdao_id_name_mapping.json", mode="r") as f:
     dao_id_name = json.load(f)

 df_raw = df_raw.assign(dao_name=lambda x: x.dao_id.replace(dao_id_name))

# Summary statistics of data

## distinct N

In [12]:
df_N = (
    df_raw.loc[:, ["dao_id", "member_address", "slug"]]
    .nunique()
    .to_frame("distinct N")
    .T
)
df_N.columns = ["DAO", "top voter", "NFT"]
df_N.pipe(display)

,DAO,top voter,NFT
distinct N,204,4600,28446


In [13]:
df_voter = df_raw.groupby("member_address").agg(
    N_nft_kinds=("slug", "nunique"),
    N_nft_quantity=("owned_asset_count", np.sum),
)
df_voter.head()

,N_nft_kinds,N_nft_quantity
member_address,,
0x000000de5f9e90ce604da5fd78acd6fae789ecca,3,7.0
0x00029d35cb7ae09d38037355a046791d7b5e1645,44,147.0
0x00290ffc9e9d19bda7b25c6e44d8adf55dfbf2dd,46,2132.0
0x0031ce5920cbe1e600113d14ca06ac6596fe7466,2,3.0
0x00320c624958997f6d8ec1d130a436e87a1f0b0e,1,1.0


In [14]:
df_voter_stats = (
    df_voter.assign(
        N_nft_quantity_wei=lambda x: x.N_nft_quantity.div(10**18),
        log_N_nft_quantity=lambda x: x.N_nft_quantity.map(np.log),
    )
    .describe()
    .assign(
        N_nft_quantity=lambda x: x.N_nft_quantity.map(lambda x: f"{x:.2e}"),
        N_nft_quantity_wei=lambda x: x.N_nft_quantity_wei.map(lambda x: f"{x:.2e}"),
        N_nft_kinds=lambda x: x.N_nft_kinds.map(lambda x: f"{x:.0f}"),
        log_N_nft_quantity=lambda x: x.log_N_nft_quantity.map(lambda x: f"{x:.2f}"),
    )
    .loc[
        ["mean", "std", "min", "25%", "50%", "75%", "max"],
        ["N_nft_kinds", "log_N_nft_quantity", "N_nft_quantity"],
    ]
)
df_voter_stats.columns = [
    "N(distinct NFT)",
    "log(Total NFT collections)",
    "Total NFT collections",
]
df_voter_stats.pipe(display)

,N(distinct NFT),log(Total NFT collections),Total NFT collections
mean,32,3.99,3.43e+72
std,60,6.20,2.33e+74
min,1,0.00,1.00e+00
25%,3,1.61,5.00e+00
50%,11,3.18,2.40e+01
75%,33,4.74,1.15e+02
max,758,175.45,1.58e+76


# Focus on top N NFT collections

In [15]:
top_N = 20

slug = (
    df_raw.groupby("slug")
    .owned_asset_count.sum()
    .sort_values(ascending=False)
    .to_frame()
)
slug.pipe(display)

slug_top = slug.head(top_N).index.tolist()
df = df_raw.loc[lambda x: x.slug.isin(slug_top)]
df.info()

,owned_asset_count
slug,
unidentified-contract-vzooewvccc,1.579209e+76
unidentified-contract-cdgfjphfbh,2.278085e+38
unidentified-contract-kgoq4pkam0,9.709694e+23
unidentified-contract-ytlksgcesz,7.281004e+23
unidentified-contract-kj63urtefs,5.053004e+22
...,...
friedguyz,1.000000e+00
sudoangels-limited,1.000000e+00
friendly-fire-v2,1.000000e+00


<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 165 to 187152
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dao_id             68 non-null     object 
 1   member_address     68 non-null     object 
 2   slug               68 non-null     object 
 3   owned_asset_count  68 non-null     float64
dtypes: float64(1), object(3)
memory usage: 2.7+ KB


# Data for network visualization
- Source
- Target
- weight

In [17]:
# !pip install tqdm
# !pip install pyvis

     ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     ------------------------------------- 592.8/592.8 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 133.1/133.1 kB 7.7 MB/s eta 0:00:00
  Created wheel for pyvis: filename=pyvis-0.3.0-py3-none-any.whl size=600233 sha256=e0e92e3562b7ef41ff060deb1297831f6338297ab716f366c84246f68e2e837b
  Stored in directory: c:\users\x\appdata\local\pip\cache\wheels\84\57\e8\d47479bd009ffbbf8a7e6a085d2456a823c58684a64830e0ac
Successfully built pyvis



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from itertools import permutations
from tqdm import tqdm
import networkx as nx
from pyvis.network import Network
import io, pickle

In [19]:
dir_path = "vis"

# DAO network

In [21]:
network_dao = {}
for grp, df_grp in tqdm(df.groupby("slug")):
    for p in permutations(sorted(df_grp.dao_id.unique()), 2):
        if p not in network_dao:
            network_dao[p] = 0
        network_dao[p] += 1

100%|██████████| 20/20 [00:00<00:00, 9666.52it/s]


In [22]:
df_network_dao = pd.DataFrame(
    [
        {"Source": source, "Target": target, "weight": weight}
        for (source, target), weight in network_dao.items()
    ]
)
df_network_dao.to_csv(f"{dir_path}/vis_network_dao.csv", index=False)

In [23]:
df_network_dao.info()
df_network_dao.head().pipe(display)
df_network_dao.weight.value_counts().to_frame("counts").pipe(display)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Source  710 non-null    object
 1   Target  710 non-null    object
 2   weight  710 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 16.8+ KB


,Source,Target,weight
0,0519572c-483c-4d6e-ac6d-28eda3ea1aff,11eb10d7-6493-4cfc-8255-cb0f2338fa67,1
1,0519572c-483c-4d6e-ac6d-28eda3ea1aff,219ab5ef-9345-448b-bbf2-786a33e4edd8,1
2,0519572c-483c-4d6e-ac6d-28eda3ea1aff,50633191-27ac-40e3-9f52-da100c196c90,1
3,0519572c-483c-4d6e-ac6d-28eda3ea1aff,5c76f492-c718-4c2e-b633-c9dbac1cc331,1
4,0519572c-483c-4d6e-ac6d-28eda3ea1aff,67149dec-7a5d-41f4-8e68-f75446ce47a0,1


,counts
1,704
2,6


In [34]:
G_dao = nx.from_pandas_edgelist(
    df_network_dao, source="Source", target="Target", edge_attr="weight"
)

nx.write_gexf(G_dao,f"{dir_path}/gragh_dao.gexf")
with io.open(f"{dir_path}/gragh_dao.nx", mode="wb") as f:
    pickle.dump(G_dao, f)

In [25]:
net_dao = Network(height="1200px", width="100%", notebook=True)
net_dao.repulsion()
net_dao.from_nx(G_dao)
net_dao.show(f"{dir_path}/pyvis_nx_dao.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# Voter network

In [26]:
network_voters = {}
for grp, df_grp in tqdm(df.groupby("slug")):
    for p in permutations(sorted(df_grp.member_address.unique()), 2):
        if p not in network_voters:
            network_voters[p] = 0
        network_voters[p] += 1

100%|██████████| 20/20 [00:00<00:00, 20887.97it/s]


In [27]:
df_network_voter = pd.DataFrame(
    [
        {"Source": source, "Target": target, "weight": weight}
        for (source, target), weight in network_voters.items()
    ]
)
df_network_voter.to_csv(f"{dir_path}/vis_network_voter.csv", index=False)

In [28]:
# network_voters = {}
# for grp, df_grp in tqdm(df.groupby('slug')):
#     for p in permutations(sorted(df_grp.member_address.unique()), 2):
#         if p not in network_voters:
#             network_voters[p] = [0, '']
#         network_voters[p][0] += 1
#         network_voters[p][1] = grp

# df_network_voter = pd.DataFrame([{'Source': source, 'Target': target, 'weight': weight, 'slug': slug} for (source, target), (weight, slug) in network_voters.items()])

# # add colors
# colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
# color_map = dict(zip(df_network_voter.slug.unique(), colors))
# df_network_voter['color'] = df_network_voter.slug.map(lambda x: color_map[x])

# df_network_voter.to_csv(f'{dir_path}/vis_network_voter.csv', index = False)

In [29]:
df_network_voter.info()
df_network_voter.head().pipe(display)
df_network_voter.weight.value_counts().to_frame("counts").pipe(display)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Source  320 non-null    object
 1   Target  320 non-null    object
 2   weight  320 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 7.6+ KB


,Source,Target,weight
0,0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01,0x14bdfda5b5b829f14332a52c15129386284ce36a,1
1,0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01,0x194f0fe6d4dfbf198868b1b507cd74122fbfebcc,1
2,0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01,0x1ca48a32482ef29708e95f248e8a613f05782ed3,1
3,0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01,0x24e6d835f23b9716af984f6604625c6b074015cd,1
4,0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01,0x4194ce73ac3fbbece8ffa878c2b5a8c90333e724,1


,counts
1,320


In [30]:
df_network_voter[["Source", "Target"]] = df_network_voter[
    ["Source", "Target"]
].applymap(lambda x: x[3:9].lower())

In [35]:
G_voter = nx.from_pandas_edgelist(
    df_network_voter, source="Source", target="Target", edge_attr=True
)
nx.write_gexf(G_voter,f"{dir_path}/gragh_voter.gexf")
with io.open(f"{dir_path}/gragh_voter.nx", mode="wb") as f:
    pickle.dump(G_voter, f)

In [32]:
net_voter = Network(height="1200px", width="100%", notebook=True)
net_voter.repulsion()
net_voter.from_nx(G_voter)
net_voter.show(f"{dir_path}/pyvis_nx_voter.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# additional data

In [33]:
df_nft = pd.read_csv("data/Data_API.csv")
df_nft.info()

FileNotFoundError: [Errno 2] No such file or directory: 'data/Data_API.csv'

In [ ]:
(df_nft.describe().round(2))

In [ ]:
df_data_nunique = df_nft.loc[
    :,
    [
        "Smart_contract",
        "ID_token",
        "Transaction_hash",
        "Seller_address",
        "Seller_username",
        "Buyer_address",
        "Buyer_username",
        "Image_url_1",
        "Image_url_2",
        "Image_url_3",
        "Image_url_4",
        # 'Price_Crypto',
        "Crypto",
        # 'Price_USD',
        "Name",
        "Description",
        "Collection",
        "Market",
        "Datetime_updated",
        "Datetime_updated_seconds",
        "Permanent_link",
        "Unique_id_collection",
        "Collection_cleaned",
        "Category",
    ],
].nunique()

In [ ]:
df_data_nunique.to_frame("distinct N")

In [ ]:
for i in sorted(df_nft.Category.unique()):
    print(i)